# LALR Parser with GUI

File: ply.lex

In [1]:
import ply.lex as lex
import types
import inspect

# Override the inspect.getsourcefile function temporarily
def dummy_getsourcefile(*args, **kwargs):
    return "dummy_file"

original_getsourcefile = inspect.getsourcefile
inspect.getsourcefile = dummy_getsourcefile

# Create a dummy module to hold the lexer rules and functions
dummy_module = types.ModuleType("dummy_module")
dummy_module.__file__ = "dummy_file"  # Set a dummy file name
dummy_module.__module__ = "dummy_module"  # Set the module name

tokens = (
    'NUMBER', 
    'PLUS', 
    'MINUS',
    'MULTIPLY',
    'DIVIDE',
    'POWER',
    'LPAREN',
    'RPAREN'
)

# Regular expression rules for tokens
t_PLUS = r'\+'
t_MINUS = r'-'
t_MULTIPLY = r'\*'
t_DIVIDE = r'/'
t_POWER = r'\*\*'
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_NUMBER = r'\d+'
t_ignore = ' \t'

# Error handling function
def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

# Assign the token variables to our dummy module
dummy_module.tokens = tokens
dummy_module.t_PLUS = t_PLUS
dummy_module.t_MINUS = t_MINUS
dummy_module.t_MULTIPLY = t_MULTIPLY
dummy_module.t_DIVIDE = t_DIVIDE
dummy_module.t_POWER = t_POWER
dummy_module.t_LPAREN = t_LPAREN
dummy_module.t_RPAREN = t_RPAREN
dummy_module.t_NUMBER = t_NUMBER
dummy_module.t_ignore = t_ignore
dummy_module.t_error = t_error

# Initialize the lexer using the dummy module
lexer = lex.lex(module=dummy_module)

# Restore the original function after lexer initialization
inspect.getsourcefile = original_getsourcefile

File: ply.yacc

In [2]:
import ply.yacc as yacc
import types
import inspect

# Reference the lexer tokens
tokens = dummy_module.tokens

# Operator precedence (from highest to lowest)
precedence = (
    ('right', 'POWER'),         # Exponentiation
    ('left', 'MULTIPLY', 'DIVIDE'),
    ('left', 'PLUS', 'MINUS'),
    ('left', 'LPAREN', 'RPAREN')
)

# Override the inspect.getsourcefile function temporarily
def dummy_getsourcefile(*args, **kwargs):
    return "dummy_file"

original_getsourcefile = inspect.getsourcefile
inspect.getsourcefile = dummy_getsourcefile

# Create a dummy module for the parser
parser_module = types.ModuleType("parser_module")
parser_module.__file__ = "dummy_file"  # Set a dummy file name

# Parsing rules
def p_expression_plus(p):
    'expression : expression PLUS expression'
    p[0] = p[1] + p[3]

def p_expression_minus(p):
    'expression : expression MINUS expression'
    p[0] = p[1] - p[3]

def p_expression_multiply(p):
    'expression : expression MULTIPLY expression'
    p[0] = p[1] * p[3]

def p_expression_divide(p):
    'expression : expression DIVIDE expression'
    p[0] = p[1] / p[3]

def p_expression_power(p):
    'expression : expression POWER expression'
    p[0] = p[1] ** p[3]

def p_expression_parenthesis(p):
    'expression : LPAREN expression RPAREN'
    p[0] = p[2]

def p_expression_number(p):
    'expression : NUMBER'
    p[0] = int(p[1])

def p_error(p):
    print("Syntax error in input!")

# Assign rules to the dummy module
parser_module.tokens = tokens
parser_module.precedence = precedence
parser_module.p_expression_plus = p_expression_plus
parser_module.p_expression_minus = p_expression_minus
parser_module.p_expression_multiply = p_expression_multiply
parser_module.p_expression_divide = p_expression_divide
parser_module.p_expression_power = p_expression_power
parser_module.p_expression_parenthesis = p_expression_parenthesis
parser_module.p_expression_number = p_expression_number
parser_module.p_error = p_error

# Build the parser
parser = yacc.yacc(module=parser_module)

# Restore the original function after parser initialization
inspect.getsourcefile = original_getsourcefile

GUI Implementation

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox, font

def on_calculate():
    input_text = input_entry.get()
    try:
        result = parser.parse(input_text)
        output_label.config(text=f"Result: {result}")
    except Exception as e:
        error_message = str(e)  # Capture the actual error message
        output_label.config(text=f"Error: {error_message}")
        messagebox.showerror("Error", error_message)

def p_error(p):
    if p:
        raise ValueError(f"Syntax error at '{p.value}'")
    else:
        raise ValueError("Syntax error at EOF")

def on_clear():
    input_entry.delete(0, tk.END)  # Clear the input field
    output_label.config(text="")   # Clear the output label

app = tk.Tk()
app.title("LALR Parser")
app.geometry("600x300")
app.configure(bg='#225ca8')  # Background color

# Styling
style = ttk.Style()
style.configure('TButton', background='#3498DB', foreground='white', font=('Arial', 12))
style.configure('TLabel', background='#2C3E50', foreground='#ECF0F1', font=('Arial', 12))
style.configure('TFrame', background='#2C3E50')

frame = ttk.Frame(app, padding="20")
frame.pack(padx=20, pady=20)

input_label = ttk.Label(frame, text="Enter Expression:")
input_label.pack(pady=5)

input_entry = ttk.Entry(frame, font=('Arial', 12), width=30)
input_entry.pack(pady=10)
input_entry.focus()

calculate_button = ttk.Button(frame, text="Calculate", command=on_calculate)
calculate_button.pack(pady=5)

clear_button = ttk.Button(frame, text="Clear", command=on_clear)
clear_button.pack(pady=5)

output_label = ttk.Label(frame, text="", font=('Arial', 12, 'bold'))
output_label.pack(pady=20)

app.mainloop()

In [4]:
# Input: 3 * (2 + 4) / 2 - 1
# Output: 18

# Input:2 ++ 2
# Output: Error